In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 13:22:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/13 13:22:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/13 13:22:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# 1

    Read the case, department, and source data into their own spark dataframes.

    Let's see how writing to the local disk works in spark:
        Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
        Inspect your folder structure. What do you notice?

    Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


df = spark.read.csv("data/source.csv", sep=",", header=True, inferSchema=True)

OR

(
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("data/source.csv")
)


In [2]:
# Read

In [3]:
case = spark.read.csv('case.csv', sep=",", header=True, inferSchema=True)

source = spark.read.csv('source.csv', sep=",", header=True, inferSchema=True)

dept = spark.read.csv('dept.csv', sep=",", header=True, inferSchema=True)




In [4]:
case.show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [5]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [6]:
dept.show(5, vertical = True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
-RECORD 2--------------------------------------
 dept_division          | Clean and Green      
 dept_name              | Parks and Recreation 
 standardized_dept_name | Parks & Recreation   
 dept_subject_to_SLA    | YES                  
-RECORD 3--------------------------------------
 dept_division          | Clean and Green N... 
 dept_name              | Parks and Recreation 
 standardized_dept_name | Parks & Recreation   
 dept_subject_to_SLA    | YES                  
-RECORD 4-------------------------------

from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("dept_division", StringType()),
        StructField("dept_name", StringType()),
        StructField("standardized_dept_name", StringType()),
        StructField("dept_subject_to_SLA", StringType())
    ]
)

spark.read.csv("dept.csv", header=True, schema=schema)


dept_division,dept_name,standardized_dept_name,dept_subject_to_SLA

# Write

mpg.write.json("data/mpg_json", mode="overwrite")

- like much else in spark, there's multiple ways we could do this:
(
    mpg.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("data/mpg_csv")
)

In [7]:
source.write.json('source_json', mode = 'overwrite')

In [8]:
case.write.json('case_json', mode = 'overwrite')

In [9]:
dept.write.json('dept_json', mode = 'overwrite')

In [10]:
source.write.csv('source_csv', mode = 'overwrite')

In [11]:
case.write.csv('case_csv', mode = 'overwrite')

In [12]:
dept.write.csv('dept_csv', mode = 'overwrite')

In [13]:
# observation: They appeared as folders and were imported in parts

In [14]:
case.printSchema() #dates are still strings, case_late and case_closed are boolean columns

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [15]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [16]:
dept.printSchema() #might wanna make yes/no column a 0/1 column

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [17]:
# above are my notes, what I would do. The first few lines below are what the curriculum directs me to do. (#c)

In [18]:
#c
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [19]:
#c
# demonstrating we only have yes/no in each field
case.groupBy("case_closed", "case_late").count().show()


+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [20]:
#c
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

case.select("case_closed", "case_late").show(5)


+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [21]:
# c
# The council_district column appears as though it is an integer, 
#but this is just a unique identifier for each district
case = case.withColumn("council_district", col("council_district").cast("string"))


In [22]:
# c
print("--- Before handling dates")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)


--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:

In [23]:
# c
# Using the trim and lower functions lets us strip any leading or trailing whitespace 
#and convert everything to lowercase.
print("--- Before")
case.select("request_address").show(5)

case = case.withColumn("request_address", trim(lower(case.request_address)))

print("--- After")
case.select("request_address").show(5)


--- Before
+--------------------+
|     request_address|
+--------------------+
|2315  EL PASO ST,...|
|2215  GOLIAD RD, ...|
|102  PALFREY ST W...|
|114  LA GARDE ST,...|
|734  CLEARVIEW DR...|
+--------------------+
only showing top 5 rows

--- After
+--------------------+
|     request_address|
+--------------------+
|2315  el paso st,...|
|2215  goliad rd, ...|
|102  palfrey st w...|
|114  la garde st,...|
|734  clearview dr...|
+--------------------+
only showing top 5 rows



# notes
how to add leading zeros to concil columm

df = df.withColumn("council_district", col("council_district").cast("int"))

 '%03d' means at least 3 digits, pad with 0s

 In order to use the format_string function the way we are, we'll need to
 convert council_district back to an integer temporarily, but the final output
 will be a string.

df = df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

df.select("council_district").show(5)


In [24]:
# c
# set zipcode column
case = case.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))

case.select("zipcode").show(5)



+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
|  78223|
|  78228|
+-------+
only showing top 5 rows



In [25]:
#  Join w/ dept dataset

In [26]:
# c
comb = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

comb.show(2, vertical=True)


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 zipcode              | 78207                
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-01 00:46:00  
 case_due_date        | 2018-01-01

In [27]:
# train test split

train, test = comb.randomSplit([0.8, 0.2])
#OR
train, validate, test = comb.randomSplit([0.6, 0.2, 0.2])



    1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
    2. How many Stray Animal cases are there?

    3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

    4. Convert the council_district column to a string column.
    5. Extract the year from the case_closed_date column.

    6. Convert num_days_late from days to hours in new columns num_hours_late.

    7. Join the case data with the source and department data.

    8. Are there any cases that do not have a request source?

    9. What are the top 10 service request types in terms of number of requests?
    10. What are the top 10 service request types in terms of average days late?
    11. Does number of days late depend on department?
    12. How do number of days late depend on department and request type?

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [28]:
# 1. How old is the latest (in terms of days past SLA) currently open issue? 
# How long has the oldest (in terms of days since opened) currently opened issue been open?

In [ ]:
# got from review

In [69]:
case[case.case_status == 'Open'].show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | mt13131              
 case_id              | 1014128388           
 case_opened_date     | 2018-01-02 09:39:00  
 case_closed_date     | 2018-01-02 09:39:00  
 case_due_date        | 2018-01-02 09:39:00  
 case_late            | true                 
 num_days_late        | 211.5974884          
 case_closed          | false                
 service_request_type | Complaint            
 SLA_days             | 7.0                  
 case_status          | Open                 
 request_address      | 7326  westglade p... 
 council_district     | 6                    
 zipcode              | 78227                
 department           | Customer Service     
 dept_subject_to_SLA  | true                 
 source_username      | Mary A Torres        
only showing top 1 row



In [70]:
case[case.case_closed == 'false'].show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | mt13131              
 case_id              | 1014128388           
 case_opened_date     | 2018-01-02 09:39:00  
 case_closed_date     | 2018-01-02 09:39:00  
 case_due_date        | 2018-01-02 09:39:00  
 case_late            | true                 
 num_days_late        | 211.5974884          
 case_closed          | false                
 service_request_type | Complaint            
 SLA_days             | 7.0                  
 case_status          | Open                 
 request_address      | 7326  westglade p... 
 council_district     | 6                    
 zipcode              | 78227                
 department           | Customer Service     
 dept_subject_to_SLA  | true                 
 source_username      | Mary A Torres        
only showing top 1 row



In [72]:
case[case.case_closed == 'false'].select('case_opened_date', 'case_due_date', 'case_closed').show(5)

+-------------------+-------------------+-----------+
|   case_opened_date|      case_due_date|case_closed|
+-------------------+-------------------+-----------+
|2018-01-02 09:39:00|2018-01-02 09:39:00|      false|
|2018-01-02 10:49:00|2018-01-02 10:49:00|      false|
|2018-01-02 13:45:00|2018-01-02 13:45:00|      false|
|2018-01-02 14:09:00|2018-01-02 14:09:00|      false|
|2018-01-02 14:34:00|2018-01-02 14:34:00|      false|
+-------------------+-------------------+-----------+
only showing top 5 rows



In [73]:
case[case.case_closed == 'false']\
    .select('case_opened_date', 'case_due_date', 'num_days_late')\
    .sort(desc('num_days_late'))\
    .show(5)

+-------------------+-------------------+------------------+
|   case_opened_date|      case_due_date|     num_days_late|
+-------------------+-------------------+------------------+
|2017-01-01 13:57:00|2017-01-01 13:57:00|       348.6458333|
|2017-01-01 13:48:00|2017-01-01 13:48:00|       348.6458333|
|2017-01-02 11:26:00|2017-01-02 11:26:00|348.52356480000003|
|2017-01-03 10:01:00|2017-01-03 10:01:00|347.58256939999995|
|2017-01-05 14:39:00|2017-01-05 14:39:00|       345.3894213|
+-------------------+-------------------+------------------+
only showing top 5 rows



In [74]:
case[case.case_closed == 'false']\
    .select('case_opened_date', 'case_due_date', 'SLA_days')\
    .sort(desc('SLA_days'))\
    .show(5)

+-------------------+-------------------+-----------+
|   case_opened_date|      case_due_date|   SLA_days|
+-------------------+-------------------+-----------+
|2017-09-22 08:27:00|2017-09-22 08:27:00| 1419.00191|
|2017-11-03 14:05:00|2017-11-03 14:05:00|     1419.0|
|2017-11-21 06:57:00|2017-11-21 06:57:00|1416.063981|
|2017-12-03 12:59:00|2017-12-03 12:59:00|1414.813009|
|2017-01-06 08:24:00|2017-01-06 08:24:00|1406.003623|
+-------------------+-------------------+-----------+
only showing top 5 rows



In [ ]:
#from review

In [59]:
case.filter('! case_closed')\
    .select('case_opened_date', 'case_due_date', 'num_days_late')\
    .sort(desc('num_days_late'))\
    .show(5)

+-------------------+-------------------+------------------+
|   case_opened_date|      case_due_date|     num_days_late|
+-------------------+-------------------+------------------+
|2017-01-01 13:57:00|2017-01-01 13:57:00|       348.6458333|
|2017-01-01 13:48:00|2017-01-01 13:48:00|       348.6458333|
|2017-01-02 11:26:00|2017-01-02 11:26:00|348.52356480000003|
|2017-01-03 10:01:00|2017-01-03 10:01:00|347.58256939999995|
|2017-01-05 14:39:00|2017-01-05 14:39:00|       345.3894213|
+-------------------+-------------------+------------------+
only showing top 5 rows



In [60]:
case.filter('! case_closed')\
    .select('case_opened_date', 'case_due_date', 'SLA_days')\
    .sort(desc('SLA_days'))\
    .show(5)

+-------------------+-------------------+-----------+
|   case_opened_date|      case_due_date|   SLA_days|
+-------------------+-------------------+-----------+
|2017-09-22 08:27:00|2017-09-22 08:27:00| 1419.00191|
|2017-11-03 14:05:00|2017-11-03 14:05:00|     1419.0|
|2017-11-21 06:57:00|2017-11-21 06:57:00|1416.063981|
|2017-12-03 12:59:00|2017-12-03 12:59:00|1414.813009|
|2017-01-06 08:24:00|2017-01-06 08:24:00|1406.003623|
+-------------------+-------------------+-----------+
only showing top 5 rows



In [29]:
# 2. How many Stray Animal cases are there?

In [30]:
case.filter(case.service_request_type == lit('Stray Animal')).count()

26760

In [31]:
#from review-also looked at
case.groupby('service_request_type')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

+---------------------------------+-----+
|service_request_type             |count|
+---------------------------------+-----+
|No Pickup                        |86855|
|Overgrown Yard/Trash             |65895|
|Bandit Signs                     |32910|
|Damaged Cart                     |30338|
|Front Or Side Yard Parking       |28794|
|Stray Animal                     |26760|
|Aggressive Animal(Non-Critical)  |24882|
|Cart Exchange Request            |22024|
|Junk Vehicle On Private Property |21473|
|Pot Hole Repair                  |20616|
|Alley-Way Maintenance            |20214|
|Lost/Stolen Cart                 |18731|
|Right Of Way/Sidewalk Obstruction|17699|
|Dead Animal - Dog                |16714|
|Cart Delivery                    |15471|
|Dead Animal - Cat                |14983|
|Animal Neglect                   |13441|
|Dead Animal - Misc               |13234|
|Trapped/Confined Animal          |11354|
|Public Nuisance(Own Animal)      |10715|
+---------------------------------

In [32]:
# 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [33]:
case.filter(case.dept_division == 'Field Operations')\
    .filter(case.service_request_type != 'Officer Standby')\
    .count()

113902

In [34]:
# 4. Convert the council_district column to a string column.

In [35]:
# i did this above, but we can verify it again
case = case.withColumn('council_district', col('council_district').cast('string'))


In [36]:
# 5. Extract the year from the case_closed_date column.

In [37]:
case.withColumn('year', year('case_closed_date'))\
    .select('service_request_type', 'case_closed_date', 'year')\
    .show(5)

+--------------------+-------------------+----+
|service_request_type|   case_closed_date|year|
+--------------------+-------------------+----+
|        Stray Animal|2018-01-01 00:42:00|2018|
|Removal Of Obstru...|2018-01-01 00:46:00|2018|
|Removal Of Obstru...|2018-01-01 00:48:00|2018|
|Front Or Side Yar...|2018-01-01 01:29:00|2018|
|Animal Cruelty(Cr...|2018-01-01 01:34:00|2018|
+--------------------+-------------------+----+
only showing top 5 rows



In [38]:
# 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [39]:
case.withColumn('num_hours_late', round(expr('num_days_late * 24'), 1))\
    .select('num_days_late', 'num_hours_late')\
    .show(10)

+-------------------+--------------+
|      num_days_late|num_hours_late|
+-------------------+--------------+
| -998.5087616000001|      -23964.2|
|-2.0126041669999997|         -48.3|
|       -3.022337963|         -72.5|
|       -15.01148148|        -360.3|
|0.37216435200000003|           8.9|
|       -29.74398148|        -713.9|
|       -14.70673611|        -353.0|
|       -14.70662037|        -353.0|
|       -14.70662037|        -353.0|
|       -14.70649306|        -353.0|
+-------------------+--------------+
only showing top 10 rows



In [40]:
case = case.withColumn('num_hours_late', round(expr('num_days_late * 24'), 1))

In [41]:
# 7. Join the case data with the source and department data.

In [42]:
#comb = case + dept
# now do:
# combo = comb + source

#set case = combo

In [43]:
comb.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 zipcode              | 78207                
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [44]:
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [45]:
combo = (
    comb
    # left join on dept_division
    .join(source, "source_id", "left")
)

combo.show(2, vertical=True)


-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 zipcode              | 78207                
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS             
-RECORD 1------------------------------------
 source_id            | svcCRMSS             
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

In [46]:
case = combo

In [47]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 zipcode              | 78207                
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS             
only showing top 1 row



In [48]:
# 8. Are there any cases that do not have a request source?

In [49]:
case.filter(case.source_id == 'null').count()

0

In [50]:
case.filter(case.source_id == '').count()

0

In [51]:
# 9. What are the top 10 service request types in terms of number of requests? #modeled after review

In [52]:
case.groupBy('service_request_type')\
    .count()\
    .sort(desc('count'))\
    .show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



In [53]:
# 10. What are the top 10 service request types in terms of average days late?

In [54]:
case.groupBy('service_request_type')\
    .agg(round(mean('num_days_late'), 1).alias('avg_late'))\
    .sort(desc('avg_late'))\
    .show(10)

+--------------------+--------+
|service_request_type|avg_late|
+--------------------+--------+
|  Zoning: Junk Yards|   176.0|
|Labeling for Used...|   162.4|
|Record Keeping of...|   154.0|
|Signage Requied f...|   151.6|
|Storage of Used M...|   142.1|
|Zoning: Recycle Yard|   135.9|
|Donation Containe...|   131.8|
|License Requied U...|   128.8|
|Traffic Signal Gr...|   101.8|
|           Complaint|    72.9|
+--------------------+--------+
only showing top 10 rows



In [55]:
# 11. Does number of days late depend on department?

In [56]:
case.groupBy('department')\
    .agg(round(mean('num_days_late'), 1).alias('avg_late'))\
    .sort(desc('avg_late'))\
    .show(10)

+--------------------+--------+
|          department|avg_late|
+--------------------+--------+
|    Customer Service|    59.7|
|         Solid Waste|    -2.2|
|        Metro Health|    -4.9|
|  Parks & Recreation|    -5.3|
|Trans & Cap Impro...|   -20.6|
|DSD/Code Enforcement|   -38.4|
|Animal Care Services|  -226.5|
|        City Council|    null|
+--------------------+--------+



In [57]:
# 12. How do number of days late depend on department and request type?

In [58]:
case.groupBy('department', 'service_request_type')\
    .agg(round(avg('num_days_late'), 1).alias('avg_days_late'))\
    .sort('department')\
    .show(50)

+--------------------+--------------------+-------------+
|          department|service_request_type|avg_days_late|
+--------------------+--------------------+-------------+
|Animal Care Services|Animal Bite(Non-C...|         -2.5|
|Animal Care Services|Spay/Neuter Reque...|         -7.0|
|Animal Care Services|Animal Bite(Criti...|          0.0|
|Animal Care Services|Animal Cruelty(Cr...|         -0.1|
|Animal Care Services|Public Nuisance(O...|         -2.2|
|Animal Care Services|Aggressive Animal...|         16.7|
|Animal Care Services|Trapped/Confined ...|          0.2|
|Animal Care Services|      Animal Neglect|          7.3|
|Animal Care Services|     Officer Standby|         -0.1|
|Animal Care Services|City Council Anim...|         -1.5|
|Animal Care Services|Animal Permits Re...|         22.2|
|Animal Care Services|Aggressive Animal...|          2.6|
|Animal Care Services|Injured Animal(Cr...|         -0.1|
|Animal Care Services|        Stray Animal|       -998.8|
|        City 